In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import RDConfig
import os
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw

In [3]:
from rdkit.Chem import PyMol
import py3Dmol
from rdkit.Chem import rdDistGeom
from rdkit.Chem import rdMolAlign

In [4]:
mol = Chem.MolFromSmiles("COC1=CC(C(C2=C3C=NC=C2)=CN(C3=O)C)=CC(OC)=C1CN4CCN(CC4)CCOCCOCCOC5=CC(C6=C(N=CS6)C)=CC=C5CNC([C@@H]7C[C@H](CN7C([C@@H](NC(C8(CC8)F)=O)C(C)(C)C)=O)O)=O")

In [5]:
mol2 = Chem.AddHs(mol)
params = AllChem.ETKDGv3()
cids = AllChem.EmbedMultipleConfs(mol2, 25, params)
res = AllChem.MMFFOptimizeMoleculeConfs(mol2)

In [6]:
print(mol2.GetNumConformers())

25


In [7]:
IPythonConsole.drawMol3D(mol2, confId=2)
IPythonConsole.drawMol3D(mol2, confId=3)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [53]:
def drawit(m, cids=[-1], p=None, removeHs=False, s=True,
           colors=('grayCarbon','grayCarbon','grayCarbon','magentaCarbon','whiteCarbon','purpleCarbon')):
        if removeHs:
            m = Chem.RemoveHs(m)
        if p is None:
            p = py3Dmol.view(width=400, height=400)
        p.removeAllModels()
        if s==False:
            for i,cid in enumerate(cids):
                IPythonConsole.addMolToView(m,p,confId=cid)
                p.addSurface(py3Dmol.VDW,{'opacity':0.5,'model':10,'color':'gray'})
                IPythonConsole.drawOptions.highlightRadius = 0.5
            for i,cid in enumerate(cids):
                #print(cid)
                IPythonConsole.addMolToView(m,p,confId=1)
                p.addSurface(py3Dmol.VDW,{'opacity':0.3,'model':10,'color':'gray'})
        for i,cid in enumerate(cids):
            p.setStyle({'model':i,},
                            {'stick':{'colorscheme':colors[i%len(colors)]}})
        if s:
            print(i)
        p.zoomTo()
        return p.show()

In [60]:
# Align by substructure (COCCOCC)
core = Chem.MolFromSmiles('COCCOCC')
rdMolAlign.AlignMolConformers(mol2,atomIds = mol2.GetSubstructMatch(core))
#rdMolAlign.AlignMol(mol2,mol2,prbCid=cids[1],refCid=cids[5])
#rdMolAlign.AlignMol(mol2,mol2,prbCid=cids[5],refCid=cids[9])
drawit(mol2,cids=[1,2,3,4], s=False, removeHs=True)
drawit(mol2,cids=[1], s=False, removeHs=False)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
m3d = mol2 
maxd = -1
for j in range(0,5):
    for i in range(j,len(cids)):
        d1 = rdMolAlign.AlignMol(m3d,m3d,prbCid=cids[i],refCid=cids[j])
        d2 = rdMolAlign.GetBestRMS(m3d,m3d,prbId=cids[i],refId=cids[j])
        delt = d1-d2
        if delt<-1e-5:
            print(f'ooops, {i}, {delt}')
        if delt>maxd:
            maxd = delt
            maxi = i
            maxj = j
d1 = rdMolAlign.AlignMol(m3d,m3d,prbCid=cids[maxi],refCid=cids[maxj])
d2 = rdMolAlign.GetBestRMS(m3d,m3d,prbId=cids[maxi],refId=cids[maxj])
print(maxi,maxd,d1,d2)